In [1]:
import matplotlib
matplotlib.use("Agg")
 
# import the necessary packages
from sklearn.preprocessing import LabelBinarizer
from sklearn.model_selection import train_test_split
from sklearn.metrics import classification_report
from keras.models import Sequential
from keras.layers.core import Dense
from keras.optimizers import SGD
from imutils import paths
import matplotlib.pyplot as plt
import numpy as np
import argparse
import random
import pickle
import cv2
from PIL import Image
import os

Using TensorFlow backend.
/home/tejasshinde/.local/lib/python3.6/site-packages/requests/__init__.py:91: RequestsDependencyWarning: urllib3 (1.25.3) or chardet (3.0.4) doesn't match a supported version!
  RequestsDependencyWarning)


In [2]:
data = []
labels = []

In [3]:
imagePaths = sorted(list(paths.list_images('images/')))
random.seed(42)
random.shuffle(imagePaths)

In [4]:
imagePaths[:5]

['images/bulbasaur/00000016.png',
 'images/squirtle/00000041.jpg',
 'images/pikachu/00000010.png',
 'images/pikachu/00000095.jpg',
 'images/pikachu/00000149.jpg']

## Resize, Flatten and append to data,label lists
### Flattening: 32x32x3 = 3072

In [5]:
for imagePath in imagePaths:
    image = cv2.imread(imagePath)
    # flatten the image into 32x32x3=3072
    image = cv2.resize(image, (32, 32)).flatten()
    data.append(image)
 
    label = imagePath.split(os.path.sep)[-2]
    labels.append(label)

In [6]:
data[:1][0]

array([0, 0, 0, ..., 0, 0, 0], dtype=uint8)

## Scale the raw pixel intensities to the range [0, 1] from [0, 255]

In [7]:
data = np.array(data, dtype="float") / 255.0
labels = np.array(labels)

## Train, Test split of 75, 25

In [8]:
(trainX, testX, trainY, testY) = train_test_split(data,
    labels, test_size=0.25, random_state=42)

In [9]:
lb = LabelBinarizer()
trainY = lb.fit_transform(trainY)
testY = lb.transform(testY)

In [10]:
model = Sequential()
model.add(Dense(1024, input_shape=(3072,), activation="sigmoid"))
model.add(Dense(512, activation="sigmoid")) 
model.add(Dense(256, activation="sigmoid")) 
# model.add(Dense(128, activation="sigmoid")) 
model.add(Dense(5, activation="softmax")) 

In [11]:
INIT_LR = 0.1
EPOCHS = 90

print("[INFO] training network...")
opt = SGD(lr=INIT_LR)
model.compile(loss="categorical_crossentropy", optimizer=opt,
	metrics=["accuracy"])

[INFO] training network...


In [12]:
H = model.fit(trainX, trainY, validation_data=(testX, testY),
	epochs=EPOCHS, batch_size=5)

Train on 720 samples, validate on 241 samples
Epoch 1/90
720/720 [==============================] - 3s 4ms/step - loss: 1.8024 - acc: 0.2222 - val_loss: 1.6334 - val_acc: 0.1577
Epoch 2/90
720/720 [==============================] - 3s 4ms/step - loss: 1.6185 - acc: 0.2319 - val_loss: 1.5960 - val_acc: 0.2407
Epoch 3/90
720/720 [==============================] - 3s 4ms/step - loss: 1.5926 - acc: 0.2611 - val_loss: 1.5798 - val_acc: 0.2199
Epoch 4/90
720/720 [==============================] - 3s 4ms/step - loss: 1.5059 - acc: 0.2917 - val_loss: 1.4161 - val_acc: 0.2988
Epoch 5/90
720/720 [==============================] - 2s 3ms/step - loss: 1.2969 - acc: 0.3875 - val_loss: 1.0983 - val_acc: 0.4481
Epoch 6/90
720/720 [==============================] - 3s 4ms/step - loss: 1.2007 - acc: 0.4153 - val_loss: 1.1324 - val_acc: 0.3402
Epoch 7/90
720/720 [==============================] - 2s 3ms/step - loss: 1.1363 - acc: 0.4528 - val_loss: 1.2817 - val_acc: 0.3817
Epoch 8/90
720/720 [==========

720/720 [==============================] - 2s 3ms/step - loss: 0.0924 - acc: 0.9667 - val_loss: 0.3936 - val_acc: 0.8880
Epoch 63/90
720/720 [==============================] - 2s 3ms/step - loss: 0.1709 - acc: 0.9542 - val_loss: 0.6360 - val_acc: 0.8299
Epoch 64/90
720/720 [==============================] - 2s 3ms/step - loss: 0.1232 - acc: 0.9611 - val_loss: 0.7434 - val_acc: 0.7967
Epoch 65/90
720/720 [==============================] - 2s 3ms/step - loss: 0.1839 - acc: 0.9403 - val_loss: 0.4687 - val_acc: 0.8755
Epoch 66/90
720/720 [==============================] - 3s 3ms/step - loss: 0.1367 - acc: 0.9514 - val_loss: 0.4917 - val_acc: 0.8631
Epoch 67/90
720/720 [==============================] - 2s 3ms/step - loss: 0.1634 - acc: 0.9486 - val_loss: 0.4072 - val_acc: 0.8963
Epoch 68/90
720/720 [==============================] - 2s 3ms/step - loss: 0.1249 - acc: 0.9611 - val_loss: 0.4715 - val_acc: 0.8755
Epoch 69/90
720/720 [==============================] - 2s 3ms/step - loss: 0.0839

In [13]:
# H.history['val_acc'][-4]
observation_list = [EPOCHS,INIT_LR, H.history['loss'][-4], H.history['val_loss'][-4], H.history['acc'][-4], H.history['val_acc'][-4]]
observation_list
with open('./observations/observation_model.txt', 'a') as file_out:
    file_out.write(str(observation_list) + '\n')

## CNN


### Conv2D( number_of_filters, (size_of_filter) )

In [14]:
out = open('./observations/observation.csv' , 'w')
headers = 'EPOCH,INITIAL LEARNING RATE,BATCH SIZE,LOSS,VAL_LOSS,ACCURACY,VAL_ACC\n'
out.write(headers)

70

In [15]:
# import the necessary packages
from keras.models import Sequential
from keras.layers.normalization import BatchNormalization
from keras.layers.convolutional import Conv2D
from keras.layers.convolutional import MaxPooling2D
from keras.layers.core import Activation
from keras.layers.core import Flatten
from keras.layers.core import Dropout
from keras.layers.core import Dense
from keras import backend as K

In [16]:
class SymNet:
	@staticmethod
	def build(width, height, depth, classes):
		# initialize the model along with the input shape to be
		# "channels last" and the channels dimension itself
		model = Sequential()
		inputShape = (height, width, depth)
		chanDim = -1

		# if we are using "channels first", update the input shape
		# and channels dimension
		if K.image_data_format() == "channels_first":
			inputShape = (depth, height, width)
			chanDim = 1
        
        # CONV => RELU => POOL layer set
		model.add(Conv2D(32, (3, 3), padding="same",
			input_shape=inputShape))
		model.add(Activation("relu"))
		model.add(BatchNormalization(axis=chanDim))
		model.add(MaxPooling2D(pool_size=(2, 2)))
		model.add(Dropout(0.25))
        
        # (CONV => RELU) * 2 => POOL layer set
		model.add(Conv2D(64, (3, 3), padding="same"))
		model.add(Activation("relu"))
		model.add(BatchNormalization(axis=chanDim))
		model.add(Conv2D(64, (3, 3), padding="same"))
		model.add(Activation("relu"))
		model.add(BatchNormalization(axis=chanDim))
		model.add(MaxPooling2D(pool_size=(2, 2)))
		model.add(Dropout(0.25))    
        
        # first (and only) set of FC => RELU layers
		model.add(Flatten())
		model.add(Dense(512))
		model.add(Activation("relu"))
		model.add(BatchNormalization())
		model.add(Dropout(0.5))
        
		# softmax classifier
		model.add(Dense(5))
		model.add(Activation("softmax"))

		# return the constructed network architecture
		return model

In [17]:
# class SymNet:
# 	@staticmethod
# 	def build(width, height, depth, classes):
# 		# initialize the model along with the input shape to be
# 		# "channels last" and the channels dimension itself
# 		model = Sequential()
# 		inputShape = (height, width, depth)
# 		chanDim = -1

# 		# if we are using "channels first", update the input shape
# 		# and channels dimension
# 		if K.image_data_format() == "channels_first":
# 			inputShape = (depth, height, width)
# 			chanDim = 1
        
#         # CONV => RELU => POOL layer set
# 		model.add(Conv2D(64, (3, 3), padding="same",
# 			input_shape=inputShape))
# 		model.add(Activation("relu"))
# 		model.add(BatchNormalization(axis=chanDim))
# 		model.add(MaxPooling2D(pool_size=(2, 2)))
# 		model.add(Dropout(0.25))
        
#         # (CONV => RELU) * 2 => POOL layer set
# 		model.add(Conv2D(128, (3, 3), padding="same"))
# 		model.add(Activation("relu"))
# 		model.add(BatchNormalization(axis=chanDim))
# 		model.add(Conv2D(64, (3, 3), padding="same"))
# 		model.add(Activation("relu"))
# 		model.add(BatchNormalization(axis=chanDim))
# 		model.add(MaxPooling2D(pool_size=(2, 2)))
# 		model.add(Dropout(0.25))    
        
# 		model.add(Conv2D(128, (3, 3), padding="same"))
# 		model.add(Activation("relu"))
# 		model.add(BatchNormalization(axis=chanDim))
# 		model.add(Conv2D(64, (3, 3), padding="same"))
# 		model.add(Activation("relu"))
# 		model.add(BatchNormalization(axis=chanDim))
# 		model.add(MaxPooling2D(pool_size=(2, 2)))
# 		model.add(Dropout(0.25))    
        
        
# 		model.add(Conv2D(64, (3, 3), padding="same"))
# 		model.add(Activation("relu"))
# 		model.add(BatchNormalization(axis=chanDim))
# 		model.add(Conv2D(64, (3, 3), padding="same"))
# 		model.add(Activation("relu"))
# 		model.add(BatchNormalization(axis=chanDim))
# 		model.add(MaxPooling2D(pool_size=(2, 2)))
# 		model.add(Dropout(0.25))    
        
#         # first (and only) set of FC => RELU layers
# 		model.add(Flatten())
# 		model.add(Dense(512))
# 		model.add(Activation("relu"))
# 		model.add(BatchNormalization())
# 		model.add(Dropout(0.5))
        
# 		# softmax classifier
# 		model.add(Dense(5))
# 		model.add(Activation("softmax"))

# 		# return the constructed network architecture
# 		return model


#### Batch Normalization is used to normalize the activations of a given input volume before passing it to the next layer in the network. It has been proven to be very effective at reducing the number of epochs required to train a CNN as well as stabilizing training itself.

In [18]:

import matplotlib
matplotlib.use("Agg")

from sklearn.preprocessing import LabelBinarizer
from sklearn.model_selection import train_test_split
from sklearn.metrics import classification_report
from keras.preprocessing.image import ImageDataGenerator
from keras.optimizers import SGD
from imutils import paths
import matplotlib.pyplot as plt
import numpy as np
import argparse
import random
import pickle
import cv2
import os

symnet = SymNet()

##  Load and preprocess data

In [19]:
data = []
labels = []

imagePaths = sorted(list(paths.list_images('images/')))
random.seed(42)
random.shuffle(imagePaths)


for imagePath in imagePaths:
	image = cv2.imread(imagePath)
	image = cv2.resize(image, (64, 64))
	data.append(image)

	label = imagePath.split(os.path.sep)[-2]
	labels.append(label)


data = np.array(data, dtype="float") / 255.0
labels = np.array(labels)

## Split Data and Binarize Labels

In [20]:
(trainX, testX, trainY, testY) = train_test_split(data,
	labels, test_size=0.25, random_state=42)
# print(trainY)

lb = LabelBinarizer()
trainY = lb.fit_transform(trainY)
testY = lb.transform(testY)

## Image Augmentation

In [21]:
# construct the image generator for data augmentation
aug = ImageDataGenerator(rotation_range=30, width_shift_range=0.1,
	height_shift_range=0.1, shear_range=0.2, zoom_range=0.2,
	horizontal_flip=True, fill_mode="nearest")

# initialize our VGG-like Convolutional Neural Network (64,64,3)
model = symnet.build(width=64, height=64, depth=3,
	classes=len(lb.classes_))

## Compile and Train Model

In [23]:
INIT_LR = 0.001
EPOCHS = 75
BS = 32

print("[INFO] training network...")
opt = SGD(lr=INIT_LR, decay=INIT_LR / EPOCHS)
model.compile(loss="categorical_crossentropy", optimizer=opt,
	metrics=["accuracy"])

H = model.fit_generator(aug.flow(trainX, trainY, batch_size=BS),
	validation_data=(testX, testY), steps_per_epoch=len(trainX) // BS,
	epochs=EPOCHS)

[INFO] training network...
Epoch 1/75
22/22 [==============================] - 15s 669ms/step - loss: 2.0877 - acc: 0.3381 - val_loss: 1.1268 - val_acc: 0.5187
Epoch 2/75
22/22 [==============================] - 13s 610ms/step - loss: 1.6348 - acc: 0.4574 - val_loss: 0.8380 - val_acc: 0.6639
Epoch 3/75
22/22 [==============================] - 14s 641ms/step - loss: 1.2759 - acc: 0.5797 - val_loss: 0.6002 - val_acc: 0.7676
Epoch 4/75
22/22 [==============================] - 13s 604ms/step - loss: 1.2930 - acc: 0.5483 - val_loss: 0.5478 - val_acc: 0.7801
Epoch 5/75
22/22 [==============================] - 14s 615ms/step - loss: 1.0404 - acc: 0.6477 - val_loss: 0.5239 - val_acc: 0.7925
Epoch 6/75
22/22 [==============================] - 14s 616ms/step - loss: 1.0128 - acc: 0.6548 - val_loss: 0.5494 - val_acc: 0.7801
Epoch 7/75
22/22 [==============================] - 13s 597ms/step - loss: 0.9066 - acc: 0.7131 - val_loss: 0.5217 - val_acc: 0.7925
Epoch 8/75
22/22 [========================

22/22 [==============================] - 15s 674ms/step - loss: 0.3544 - acc: 0.8866 - val_loss: 0.4035 - val_acc: 0.8755
Epoch 63/75
22/22 [==============================] - 14s 654ms/step - loss: 0.4128 - acc: 0.8709 - val_loss: 0.2850 - val_acc: 0.9129
Epoch 64/75
22/22 [==============================] - 14s 650ms/step - loss: 0.3817 - acc: 0.8864 - val_loss: 0.2898 - val_acc: 0.9087
Epoch 65/75
22/22 [==============================] - 14s 643ms/step - loss: 0.3475 - acc: 0.8864 - val_loss: 0.3410 - val_acc: 0.9046
Epoch 66/75
22/22 [==============================] - 14s 620ms/step - loss: 0.3246 - acc: 0.9020 - val_loss: 0.2826 - val_acc: 0.9212
Epoch 67/75
22/22 [==============================] - 14s 641ms/step - loss: 0.4370 - acc: 0.8553 - val_loss: 0.2705 - val_acc: 0.9170
Epoch 68/75
22/22 [==============================] - 14s 650ms/step - loss: 0.4086 - acc: 0.8750 - val_loss: 0.4268 - val_acc: 0.8672
Epoch 69/75
22/22 [==============================] - 14s 635ms/step - loss

## Save Observations

In [24]:
# out = open('./observations/observation.csv' , 'a')
observation_list = [EPOCHS,INIT_LR,BS, H.history['loss'][-1], H.history['val_loss'][-1], H.history['acc'][-1], H.history['val_acc'][-1]]
observation_list
with open('./observations/observation.csv', 'a') as file_out:
    file_out.write(str(observation_list) + '\n')
# out.write(EPOCHS + ',' + INIT_LR + ',' + BS + ',' + H.history['loss'][-1] + ',' + H.history['val_loss'][-1] + ',' + H.history['acc'][-1] + ',' + H.history['val_acc'][-1] + '\n')
# out.write(str(observation_list))

## Plot Results

In [25]:
# evaluate the network
print("[INFO] evaluating network...")
predictions = model.predict(testX, batch_size=32)
print(classification_report(testY.argmax(axis=1),
	predictions.argmax(axis=1), target_names=lb.classes_))

N = np.arange(0, EPOCHS)
plt.style.use("ggplot")
plt.figure()
plt.plot(N, H.history["loss"], label="train_loss")
plt.plot(N, H.history["val_loss"], label="val_loss")
plt.plot(N, H.history["acc"], label="train_acc")
plt.plot(N, H.history["val_acc"], label="val_acc")
plt.title("Training Loss and Accuracy (SmallVGGNet)")
plt.xlabel("Epoch #")
plt.ylabel("Loss/Accuracy")
plt.legend()
plt.savefig('symnet_results.png')

print("[INFO] serializing network and label binarizer...")
model.save('symnet.model')
f = open('labels.bin', "wb")
f.write(pickle.dumps(lb))
f.close()

[INFO] evaluating network...
              precision    recall  f1-score   support

   bulbasaur       0.96      0.96      0.96        53
  charmander       0.89      0.92      0.90        51
      mewtwo       0.94      0.96      0.95        52
     pikachu       0.86      0.94      0.90        47
    squirtle       0.90      0.74      0.81        38

    accuracy                           0.91       241
   macro avg       0.91      0.90      0.91       241
weighted avg       0.91      0.91      0.91       241

[INFO] serializing network and label binarizer...
